### Total variation supervised learning

This code is an implementation of the Total variation model for supervised learning as described in eqns (11) and (12) in the paper:

Supervised Learning via Euler's Elastica Models
Tong Lin, Hanlin Xue, Ling Wang, Bo Huang, Hongbin Zha.
Year: 2015, Volume: 16, Issue: 111, Pages: 3637−3686

In [5]:
import autograd.numpy as np
from pylab import *
from autograd.util import *

### Computes $\phi_i(x)$

In [6]:
def phi(x,c,xi):
    dim1,dim2 = xi.shape
    return np.reshape(np.exp(-0.5*c*np.linalg.norm(x-xi, axis=1)**2),(dim1,1))

### Computes $u(x)$

In [7]:
def u(x,c,xi,w):
    dim1,dim2 = xi.shape
    return np.sum(w*np.reshape(np.exp(-0.5*c*np.linalg.norm(x-xi, axis=1)**2),(dim1,1)))

### Computes $\psi(x)$

In [8]:
def psi(x,c,xi,w):
    m=len(w)
    dim1,dim2 = x.shape
    M=np.zeros((dim1,m))
    for i in range(dim1):
        M[i,:]=np.reshape(phi(x[i],c,xi),m)
    return M

### Computes $g(x)$

In [9]:
def g(x,c,xi,w):
    return np.sum(w*(x-xi)*phi(x,c,xi), axis=0)

### The following functions compute the gradient $\nabla u(x)$, the Laplacian $\triangle u(x)$ and the Hessian $H(u(x))$ by using the definitions from the paper

In [10]:
def gradient_u(x,c,xi,w):
    return -c*g(x,c,xi,w)

In [11]:
def laplacian_u(x,c,xi,w):
    dim1,dim2 = xi.shape
    return c*np.sum(w*np.reshape((c*(np.linalg.norm(x-xi, axis=1)**2)-dim2),(dim1,1))*phi(x,c,xi))

In [12]:
def hessian_u(x,c,xi,w):
    dim1,dim2 = xi.shape
    p=-c*np.sum(w*np.reshape(np.exp(-0.5*c*np.linalg.norm(x-xi, axis=1)**2),(dim1,1)))*np.identity(dim2)
    q=np.zeros((dim2,dim2))
    pp=phi(x,c,xi)
    for k in range(len(w)):
        q=q+w[k]*(x-xi[k]).T.dot((x-xi[k]))*pp[k]
    return p+(c**2)*q

### The following functions compute the gradient $\nabla u(x)$, the Laplacian $\triangle u(x)$ and the Hessian $H(u(x))$ by using automatic differentiation

In [13]:
def autograd_gradient_u(x,c,xi,w):
    nablau= np.grad(u,0)
    return nablau(x,c,xi,w)

In [14]:
def autograd_laplacian_u(x,c,xi,w):
    h=np.autograd_hessian_u(x,c,xi,w)
    return np.matrix.trace(h)

In [15]:
def autograd_hessian_u(x,c,xi,w):
    dim1,dim2 = xi.shape
    def fun(x,c,xi,w):
        return np.sum(w*np.reshape(np.exp(-0.5*c*np.linalg.norm(x-xi, axis=1)**2),(dim1,1)))
    hess = np.hessian(fun)
    return np.reshape(hess(x,c,xi,w),(dim2,dim2))

### Computes one entry of $\partial u / \partial t$

In [16]:
def dudtv(c,xi,w,y,lambdas):
    m=len(w)
    dudtvv=np.zeros((m,1))
    for i in range(m):
        dudtvv[i,0]=dudt(xi[i],c,xi,w,y[i],lambdas)
    return dudtvv 

### Computes the vector $\partial u / \partial t$

In [17]:
def dudt(x,c,xi,w,y,lambdas):
    #here you may decide to use the automatic differentiation as above
    uu=u(x,c,xi,w)
    gu=gradient_u(x,c,xi,w)
    hu=hessian_u(x,c,xi,w)
    lu=laplacian_u(x,c,xi,w)
    ngu=np.linalg.norm(gu)
    return (lambdas*(lu-(gu.dot(hu).dot(gu.T))/(gu.dot(gu.T)))/ngu - uu + y).item()

### The following four functions are using for training and testing

In [18]:

def training_core(c,xi,yin,lambdas,tol,tau):
    dim1,dim2 = xi.shape
    w=np.random.random((dim1,1))
    PSI=psi(xi,c,xi,w)
    w=np.linalg.inv(PSI.T.dot(PSI) + eta*np.identity(dim1)).dot(PSI.T.dot(yin))
    w=np.reshape(w,(dim1,1))
    
    nr=1
    i=0
    while nr > tol:
        if i==50:
            break
        i=i+1
        PSI=psi(xi,c,xi,w)
        DUDT=dudtv(c,xi,w,yin,lambdas)  
        residual=np.linalg.inv(PSI.T.dot(PSI) + eta*np.identity(dim1)).dot(PSI.T.dot(DUDT))
        w = w + tau*residual
        nr=np.linalg.norm(residual)/len(w)
        #print('iter= %3.0i, rel.residual= %1.2e' % (i,nr))
    yout=psi(xi,c,xi,w).dot(w).T

    inds=np.where(yout > 0)
    yout[inds]=1
    inds=np.where(yout < 0)
    yout[inds]=-1
    return yout, w

In [19]:
def training_step(c,feature_training,lambdas,tol,tau,eta):
    dim1,dim2 = feature_training.shape
    x=np.zeros((1,dim2-1))

    xi=np.zeros((dim1,dim2))
    xi=feature_training[0:dim1,1:dim2]

    yin=np.zeros((dim1,1))
    yin[:,0]=feature_training[:,0]

    yout, w =training_core(c,xi,yin,lambdas,tol,tau)
    
    Error=np.matrix.trace(yout!=yin)
    Efficiency=100-100*Error/dim1
   
    return yout, w, Error, Efficiency

In [20]:
def testing_step(feature_test,c,feature_training,w):
    dim1,dim2 = feature_test.shape
    xt=np.zeros((dim1,dim2))
    xt=feature_test[0:dim1,1:dim2]

    yin=np.zeros((dim1,1))
    yin[:,0]=feature_test[:,0]

    dim1,dim2 = feature_training.shape
    xi=np.zeros((dim1,dim2))
    xi=feature_training[0:dim1,1:dim2]

    yout=psi(xt,c,xi,w).dot(w).T

    inds=np.where(yout > 0)
    yout[inds]=1
    inds=np.where(yout < 0)
    yout[inds]=-1

    dim1,dim2 = feature_test.shape
    Error=np.matrix.trace(yout!=yin)
    Efficiency=100-100*Error/dim1
   
    return yout, Error, Efficiency

In [21]:
def test_classifier(MAX,c,lambdas,eta,tol,mydata,upperbound,lowerbound,set_size):
    test_eff= np.zeros((MAX))
    train_eff=np.zeros((MAX))
    for I in range(MAX):
        random_number = int(np.random.randint(upperbound+1,size=1))
        feature_test = mydata[random_number:random_number+set_size,:]
        feature_training = np.vstack(( mydata[0:random_number,:], mydata[random_number+set_size:m,:] ))

        yout, w, train_error, train_eff[I] = training_step(c,feature_training,lambdas,tol,tau,eta)
        yout, test_error, test_eff[I]= testing_step(feature_test,c,feature_training,w)  

        inds=np.where(test_eff != 0)
        indst=inds[0]
        test_eff_mean=np.sum(test_eff[inds])/len(indst)
        print('Training_Eff = %2.2f, Testing_eff= %2.2f, Mean_Test_Eff= %2.2f' % (train_eff[I], test_eff[I], test_eff_mean))
        
        file1 = open("resultsTVfile.txt","a") 
        s =  'c= ' + repr(c) + ', lambda= ' + repr(lambdas) + ', eta= ' + repr(eta) + ', trainef= ' + repr(train_eff[I]) + ', testef= ' + repr(test_eff[I])+ ', trainmeanef= ' + repr(test_eff_mean) + '\n'
        file1.write(s) 
        file1.close() 
    return test_eff_mean

### This is the main part of the program.
A dataset for liver disorder is used for testing. Data dimension is six and labels are binary.
A search for the best parameters is done and results for each combination are saved into the file "resultsTVfile.txt"


In [ ]:
mydata2 = np.loadtxt('bupaNormalized.dat')
m,n = mydata2.shape
mydata = np.hstack ((mydata2[:,6].reshape(m,1), mydata2[:,0:6]))

N=5
set_size=int(round(m/N))
upperbound=m-set_size
lowerbound=set_size

MAX=10

cc=np.array([0.1, 0.2, 0.5, 1.0, 2.0])
lambdass=np.array([0.001, 0.01, 0.1, 1.0, 10.0, 100])
etas=np.array([.001])

# Parameter search vectors
cc = np.arange(0.1, 2, np.log(2.))
lambdass = np.arange(0.01, 1, np.log(2.))

tol=0.005
tau=0.1

file1 = open("resultsTVfile.txt","w") 
file1.write("Results for TV model - May 23, 2020 \n") 
file1.close()

for c in cc:
    for lambdas in lambdass:
        for eta in etas:
            file1 = open("resultsTVfile.txt","a") 
            s =  'processing with c= ' + repr(c) + ', lambda= ' + repr(lambdas) + ', eta= ' + repr(eta) + '\n'
            file1.write(s)
            file1.close()
            print('Processing with lambda= %1.3f, c= %2.3f, eta= %1.3f, tol= %1.3f' % (lambdas,c,eta,tol))
            test_classifier(MAX,c,lambdas,eta,tol,mydata,upperbound,lowerbound,set_size)
            
            

Processing with lambda= 0.010, c= 0.100, eta= 0.001, tol= 0.005
